In [1]:
import panel as pn  # GUI
pn.extension()

## Function that collects user and assistant messages over time

In [2]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [4]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('MTR_B2C_11-2021.csv')  # Replace with actual file path
df.head()

/var/folders/58/1hv848fx63xgbs6vs4y3p5jh0000gn/T/ipykernel_2358/267623989.py:1: DtypeWarning: Columns (1,6,7,10,73,74,75,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MTR_B2C_11-2021.csv')  # Replace with actual file path


,Seller Gstin,Invoice Number,Invoice Date,Transaction Type,Order Id,Shipment Id,Shipment Date,Order Date,Shipment Item Id,Quantity,...,Tcs Sgst Amount,Tcs Utgst Rate,Tcs Utgst Amount,Tcs Igst Rate,Tcs Igst Amount,Warehouse Id,Fulfillment Channel,Payment Method Code,Credit Note No,Credit Note Date
0,24AAACC3509K1ZX,AMD2-1000,2021-11-20 17:23:55,Shipment,407-9897385-6838725,A048755529CGB3QNUH6KJ,2021-11-22 21:27:32,2021-11-20 17:16:08,311616396930,1,...,0.00,0,0,0.01,5.92,AMD2,AFN,CC,NaN,NaN
1,24AAACC3509K1ZX,AMD2-1001,2021-11-20 19:16:41,Cancel,407-4614875-1175512,A07040712Z2Q6V11VP42T,NaN,NaN,311673515693,1,...,0.00,0,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN
2,24AAACC3509K1ZX,AMD2-1001,2021-11-20 19:16:41,Shipment,407-4614875-1175512,A07040712Z2Q6V11VP42T,2021-11-21 00:22:18,2021-11-20 15:16:02,311673515693,1,...,0.00,0,0,0.01,2.92,AMD2,AFN,CC_GC_POA,NaN,NaN
3,24AAACC3509K1ZX,AMD2-1002,2021-11-20 22:24:38,Shipment,407-0215054-1637924,A08796773KJ1EX2EG0EYZ,2021-11-21 08:44:14,2021-11-20 17:58:27,311601165287,1,...,0.00,0,0,0.01,4.28,AMD2,AFN,CC,NaN,NaN
4,24AAACC3509K1ZX,AMD2-1003,2021-11-20 23:18:13,Shipment,403-3576403-6925145,A0868045X47SCEQDLLQH,2021-11-21 16:53:55,2021-11-20 21:56:13,311678971111,1,...,1.63,0,0,0.00,0.00,AMD2,AFN,GC_POA_PayStation,NaN,NaN


In [3]:
import sqlite3

# Create SQLite in-memory DB or file-based
conn = sqlite3.connect('orders.db')  # or use ':memory:' for temporary in-memory DB

# Write to DB (replace if exists)
df.to_sql('orders', conn, if_exists='replace', index=False)

8809

In [4]:
query_df = pd.read_sql_query("""SELECT COUNT(DISTINCT "Order Id") FROM orders WHERE "Transaction Type" = 'Shipment'
    AND strftime('%Y-%m', "Invoice Date") = '2021-11'""", conn)
print(query_df)

   COUNT(DISTINCT "Order Id")
0                        7037


In [5]:
pd.read_sql_query("SELECT DISTINCT `Transaction Type` FROM orders", conn)

,Transaction Type
0,Shipment
1,Cancel
2,Refund
3,FreeReplacement


In [6]:
# Cell 1: Imports and setup
import pandas as pd
import sqlite3
import openai
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

True

In [7]:
# Load your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
# print(openai.api_key)

In [8]:
# Cell 2: Load Run Config and Questions
runs_df = pd.read_csv("runs.csv")
questions_df = pd.read_csv("questions.csv")

In [68]:
questions_df.iloc[0]

question           How many orders did I have in November 2021
SQL query    SELECT COUNT(DISTINCT "Order Id") FROM orders\...
answer                                                    7372
Name: 0, dtype: object

In [9]:
runs_df.iloc[0]

Run                       1
Model         gpt-3.5-turbo
Prompt                Basic
Tool                     No
Model Type        Execution
Name: 0, dtype: object

In [25]:
# Cell 3: Setup SQLite in-memory DB (load your orders dataset)
df = pd.read_csv("MTR_B2C_11-2021.csv", dayfirst=True)
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
df.fillna('', inplace=True)
conn = sqlite3.connect(":memory:")
df.to_sql("orders", conn, if_exists="replace", index=False)

/var/folders/58/1hv848fx63xgbs6vs4y3p5jh0000gn/T/ipykernel_97806/1465761433.py:2: DtypeWarning: Columns (1,6,7,10,73,74,75,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("MTR_B2C_11-2021.csv", dayfirst=True)
/var/folders/58/1hv848fx63xgbs6vs4y3p5jh0000gn/T/ipykernel_97806/1465761433.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


8809

In [109]:
# pd.read_sql_query("PRAGMA table_info(orders);", conn)
df.columns

Index(['Seller Gstin', 'Invoice Number', 'Invoice Date', 'Transaction Type',
       'Order Id', 'Shipment Id', 'Shipment Date', 'Order Date',
       'Shipment Item Id', 'Quantity', 'Item Description', 'Asin', 'Hsn/sac',
       'Sku', 'Product Tax Code', 'Bill From City', 'Bill From State',
       'Bill From Country', 'Bill From Postal Code', 'Ship From City',
       'Ship From State', 'Ship From Country', 'Ship From Postal Code',
       'Ship To City', 'Ship To State', 'Ship To Country',
       'Ship To Postal Code', 'Invoice Amount', 'Tax Exclusive Gross',
       'Total Tax Amount', 'Cgst Rate', 'Sgst Rate', 'Utgst Rate', 'Igst Rate',
       'Compensatory Cess Rate', 'Principal Amount', 'Principal Amount Basis',
       'Cgst Tax', 'Sgst Tax', 'Igst Tax', 'Utgst Tax',
       'Compensatory Cess Tax', 'Shipping Amount', 'Shipping Amount Basis',
       'Shipping Cgst Tax', 'Shipping Sgst Tax', 'Shipping Utgst Tax',
       'Shipping Igst Tax', 'Shipping Cess Tax Amount', 'Gift Wrap Amount'

In [10]:
# Cell 4: Function - Call LLM

def call_llm(messages, model_name="gpt-3.5-turbo", tools=None, tool_choice=None):
    response = openai.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice
    )
    return response

In [11]:
DATABASE_SCHEMA = """Here are the columns of the database table 'orders':

['Seller Gstin', 'Invoice Number', 'Invoice Date', 'Transaction Type',
       'Order Id', 'Shipment Id', 'Shipment Date', 'Order Date',
       'Shipment Item Id', 'Quantity', 'Item Description', 'Asin', 'Hsn/sac',
       'Sku', 'Product Tax Code', 'Bill From City', 'Bill From State',
       'Bill From Country', 'Bill From Postal Code', 'Ship From City',
       'Ship From State', 'Ship From Country', 'Ship From Postal Code',
       'Ship To City', 'Ship To State', 'Ship To Country',
       'Ship To Postal Code', 'Invoice Amount', 'Tax Exclusive Gross',
       'Total Tax Amount', 'Cgst Rate', 'Sgst Rate', 'Utgst Rate', 'Igst Rate',
       'Compensatory Cess Rate', 'Principal Amount', 'Principal Amount Basis',
       'Cgst Tax', 'Sgst Tax', 'Igst Tax', 'Utgst Tax',
       'Compensatory Cess Tax', 'Shipping Amount', 'Shipping Amount Basis',
       'Shipping Cgst Tax', 'Shipping Sgst Tax', 'Shipping Utgst Tax',
       'Shipping Igst Tax', 'Shipping Cess Tax Amount', 'Gift Wrap Amount',
       'Gift Wrap Amount Basis', 'Gift Wrap Cgst Tax', 'Gift Wrap Sgst Tax',
       'Gift Wrap Utgst Tax', 'Gift Wrap Igst Tax',
       'Gift Wrap Compensatory Cess Tax', 'Item Promo Discount',
       'Item Promo Discount Basis', 'Item Promo Tax',
       'Shipping Promo Discount', 'Shipping Promo Discount Basis',
       'Shipping Promo Tax', 'Gift Wrap Promo Discount',
       'Gift Wrap Promo Discount Basis', 'Gift Wrap Promo Tax',
       'Tcs Cgst Rate', 'Tcs Cgst Amount', 'Tcs Sgst Rate', 'Tcs Sgst Amount',
       'Tcs Utgst Rate', 'Tcs Utgst Amount', 'Tcs Igst Rate',
       'Tcs Igst Amount', 'Warehouse Id', 'Fulfillment Channel',
       'Payment Method Code', 'Credit Note No', 'Credit Note Date']


"""

In [36]:
BUSINESS_LOGIC= """ 
For sales or orders, the Transaction Type is 'Shipment'. 
For refunds, the Transaction Type is 'Refund'. 
For Cancellations, the Transaction Type is 'Cancel'.
For Replacements, the Transaction Type is 'FreeReplacement'
'"""

In [37]:
# Cell 5: Function - Evaluate One Question in One Run

def evaluate_run(run_config, question_row):
    run_id = run_config['Run']
    tool_enabled = str(run_config['Tool']).strip().lower() == 'yes' #Check if the value in the Tool column of the current run_config row is "Yes". If it is, tool_enabled will be set to True. Otherwise, it will be False.
    model_name = run_config['Model'] if 'Model' in run_config else "gpt-3.5-turbo"

    print(f"run id:{run_id}, tool_enabled: {tool_enabled}, model:{model_name}")

    # system_prompt = f"""You are an AI that generates SQL queries and executes them for a database with schema {{PROMPT_SCHEMA}}.
    # Use the business logic {{BUSINESS_LOGIC}}.
    # Only use user_id-specific filters where needed.
    # Answer clearly, no SQL shown to user."""

    ## basic
    # system_prompt = f"""
    #                 You are an expert business analyst that generates SQL queries and executes them for an amazon orders database. 
    #                 """
    ##basic+
    # system_prompt = f"""
    #                 You are an expert business analyst that generates SQL queries and executes them for an amazon orders database. 
    #                 This database is in sqlite format so ensure your SQL queries cater to sqlite.
    #                 When referring to column names, always use double quotes (e.g., \"Order Date\") if the column name contains spaces or special characters.
    #                 """
    ##basic++ WITH BUSINESS LOGIC
    system_prompt = f"""
                    You are an expert business analyst that generates SQL queries and executes them for an amazon orders database. 
                    This database is in sqlite format so ensure your SQL queries cater to sqlite.
                    When referring to column names, always use double quotes (e.g., \"Order Date\") if the column name contains spaces or special characters.
                    Use the business logic { BUSINESS_LOGIC }.
                    """
    ## basic++ WITH SCHEMA
    # system_prompt = f"""
    #     You are an expert business analyst that generates SQL queries and executes them for an amazon orders database with schema {{ DATABASE_SCHEMA }} . 
    #     This database is in sqlite format so ensure your SQL queries cater to sqlite.
    #     When referring to column names, always use double quotes (e.g., \"Order Date\") if the column name contains spaces or special characters.
    #     """

    print(f"System prompt: {system_prompt}")
    user_question = question_row['question']
    expected_answer = question_row['answer']

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ]

    tools = None
    if tool_enabled:
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "sql_query_generator",
                    "description": "Executes an SQL query generated by OpenAI",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "The SQL query to run"}
                        },
                        "required": ["query"]
                    }
                }
            }
        ]

    print(f"tools : {tools}")
    tool_choice = "auto" if tool_enabled else None
    response = call_llm(messages, model_name=model_name, tools=tools, tool_choice=tool_choice)
    tool_calls = response.choices[0].message.tool_calls if hasattr(response.choices[0].message, 'tool_calls') else []

    used_tool = False
    tool_sql_result = None
    summary = response.choices[0].message.content if hasattr(response.choices[0].message, 'content') else ''

    if tool_calls:
        used_tool = True
        function_args = json.loads(tool_calls[0].function.arguments)
        query = function_args['query']
        print(f"QUERY USED: {query}")

        try:
            tool_sql_result = pd.read_sql_query(query, conn)
            print(f"tool_sql_result :{tool_sql_result}")
            summary = str(tool_sql_result.iloc[0, 0]) if tool_sql_result.shape == (1, 1) else tool_sql_result.to_string(index=False)
        except Exception as e:
            summary = f"SQL ERROR: {e}"

    return {
        'run_id': run_id,
        'question': user_question,
        'query':query,
        'expected': expected_answer,
        'used_tool': used_tool,
        'response': summary,
    }

In [132]:
pd.read_sql_query("""SELECT COUNT(*) AS TotalOrders FROM orders WHERE strftime('%Y-%m', "Order Date") = '2021-11'""", conn)

,TotalOrders
0,7539


In [26]:
selected_run = runs_df[runs_df['Run'] == 3].iloc[0]

first_question = questions_df.iloc[0]

In [42]:
second_question = questions_df.iloc[1]
third_question = questions_df.iloc[2]
fourth_question = questions_df.iloc[3]
fifth_question = questions_df.iloc[4]
sixth_question = questions_df.iloc[5]
seventh_question = questions_df.iloc[6]
eighth_question = questions_df.iloc[7]
ninth_question = questions_df.iloc[8]
tenth_question = questions_df.iloc[9]

In [27]:
selected_run

Run                       3
Model         gpt-3.5-turbo
Prompt                Basic
Tool                    Yes
Model Type        Execution
Name: 2, dtype: object

In [44]:
# first_question['question']

fifth_question

question        Total how many different products were shipped
SQL query    SELECT COUNT(DISTINCT "Item Description") FROM...
answer                                                     229
Name: 4, dtype: object

In [43]:
result = evaluate_run(selected_run, fifth_question)

run id:3, tool_enabled: True, model:gpt-3.5-turbo
System prompt: 
                    You are an expert business analyst that generates SQL queries and executes them for an amazon orders database. 
                    This database is in sqlite format so ensure your SQL queries cater to sqlite.
                    When referring to column names, always use double quotes (e.g., "Order Date") if the column name contains spaces or special characters.
                    Use the business logic  
For sales or orders, the Transaction Type is 'Shipment'. 
For refunds, the Transaction Type is 'Refund'. 
For Cancellations, the Transaction Type is 'Cancel'.
For Replacements, the Transaction Type is 'FreeReplacement'
'.
                    
tools : [{'type': 'function', 'function': {'name': 'sql_query_generator', 'description': 'Executes an SQL query generated by OpenAI', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The SQL query to run'}}, 'required

In [40]:
result

{'run_id': np.int64(3),
 'question': 'How many orders got cancelled',
 'query': 'SELECT COUNT(*) AS \'Cancelled Orders\' FROM Orders WHERE "Transaction Type" = \'Cancel\';',
 'expected': '1159',
 'used_tool': True,
 'response': '1159'}

In [144]:
# Add accuracy column (numerical if possible)
# 1.0 means a perfect match
# < 1.0 reflects deviation in either direction
# No inflation above 100%

def compute_accuracy(row):
    try:
        expected = float(row['expected'])
        response = float(row['response'])
        error_margin = abs(response - expected) / expected
        return f"{round((1 - error_margin) * 100, 2)}%"  # 1.0 = perfect, lower means deviation
    except:
        return None

In [175]:
compute_accuracy(result)

'0.0%'

In [30]:
# Cell 6: Run All
# results = []

for _, run in runs_df.iterrows():
    # print(run)
    for _, q in questions_df.iterrows():
        # print (q)
        # result = evaluate_run(run, q)
        # results.append(result)

# results_df = pd.DataFrame(results)
# results_df.to_csv("results/eval_results.csv", index=False)
# results_df.head()

SyntaxError: incomplete input (1686545352.py, line 13)

In [ ]:
### RUBRIC
def eval_with_rubric(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by looking at the context that the customer service \
    agent is using to generate its response. 
    """

    user_message = f"""\
You are evaluating a submitted answer to a question based on the context \
that the agent uses to answer the question.
Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Context]: {context}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the context. \
Ignore any differences in style, grammar, or punctuation.
Answer the following questions:
    - Is the Assistant response based only on the context provided? (Y or N)
    - Does the answer include information that is not provided in the context? (Y or N)
    - Is there any disagreement between the response and the context? (Y or N)
    - Count how many questions the user asked. (output a number)
    - For each question that the user asked, is there a corresponding answer to it?
      Question 1: (Y or N)
      Question 2: (Y or N)
      ...
      Question N: (Y or N)
    - Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response